In [2]:
import os

from main import FeverLoader, PubhealthLoader, ClimateFeverLoader

## Setting
- 1: train on FEVER
- 2: train on pubhealth
- 3: train on climate

In [45]:
experiment = "1"
model_dirs = {
    "1": "../models/BERT_FEVER",
    "2": "../models/BERT_PUBHEALTH",
    "3": "../models/BERT_CLIMATE"
}

## Load Data

In [4]:
root = '../data_2023_06_02'

climate_in = os.path.join(root, 'preprocessed/CLIMATE-FEVER')
pubhealth_in = os.path.join(root, 'preprocessed/PUBHEALTH')
fever_in = os.path.join(root, 'preprocessed/FEVER')

In [5]:
fever_train_ds, fever_dev_ds, fever_test_ds = FeverLoader.load(fever_in)
print(len(fever_test_ds))
print(len(fever_dev_ds))
print(len(fever_train_ds))

9999
9999
145449


In [6]:
pubhealth_train_ds, pubhealth_dev_ds, pubhealth_test_ds = PubhealthLoader.load(pubhealth_in)
print(len(pubhealth_train_ds))
print(len(pubhealth_dev_ds))
print(len(pubhealth_test_ds))

8370
1050
1032


In [7]:
climate_ds = ClimateFeverLoader.load(climate_in)
print(len(climate_ds))

1381


## Training

In [8]:
model_name = "bert-base-uncased" #https://huggingface.co/bert-base-uncased
# model_name = "bert-large-uncased" #https://huggingface.co/bert-large-uncased
# model_name = "allenai/scibert_scivocab_uncased" #https://huggingface.co/allenai/scibert_scivocab_uncased

#### Set up training data
- ds1: train on FEVER
- ds2: train on pubhealth
- ds3: train on climate

In [9]:
from datasets import Dataset, DatasetDict, ClassLabel, Value, Features

/scratch/users/k21193529/conda/jenv3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
features = Features({
    "claim": Value("string"), 
    "evidence": Value("string"),
    "label": ClassLabel(num_classes=3, names=["SUPPORTS", "REFUTES", "NOT ENOUGH INFO"])
})

In [11]:
# train on fever 
ds1 = DatasetDict()
ds1['train'] = Dataset.from_list(fever_train_ds, features=features)
ds1['validation'] = Dataset.from_list(fever_dev_ds, features=features)
ds1['fever_test'] = Dataset.from_list(fever_test_ds, features=features)
ds1['pubhealth_test'] = Dataset.from_list(pubhealth_test_ds, features=features)
ds1['climate_test']  = Dataset.from_list(climate_ds, features=features)

In [12]:
# train on pubhealth
ds2 = DatasetDict()
ds2['train'] = Dataset.from_list(pubhealth_train_ds, features=features)
ds2['validation'] = Dataset.from_list(pubhealth_dev_ds, features=features)
ds2['fever_test'] = Dataset.from_list(fever_test_ds, features=features)
ds2['pubhealth_test'] = Dataset.from_list(pubhealth_test_ds, features=features)
ds2['climate_test']  = Dataset.from_list(climate_ds, features=features)

In [13]:
# train on climate
ds3 = DatasetDict()

#split climate data
from sklearn.model_selection import train_test_split

climate_train_ds, climate_dev_ds = train_test_split(climate_ds, random_state=392)
print(len(climate_train_ds))
print(len(climate_dev_ds))


ds3['train'] = Dataset.from_list(climate_train_ds, features=features)
ds3['validation'] = Dataset.from_list(climate_dev_ds, features=features)
ds3['fever_test'] = Dataset.from_list(fever_test_ds, features=features)
ds3['pubhealth_test'] = Dataset.from_list(pubhealth_test_ds, features=features)

1035
346


#### Tokenize data

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
def preprocess_function(samples):
    return tokenizer(samples['claim'], samples['evidence'], 
                     padding=True,
                     truncation='only_second', 
                     max_length=512)

In [46]:
if experiment == "1":
    ds = ds1
elif experiment == "2":
    ds = ds2
elif experiment == "3":
    ds = ds3
else:
    raise ValueError("Unknown Experiment")

In [47]:
encoded_ds = ds.map(preprocess_function, batched=True)

In [48]:
print(encoded_ds)

DatasetDict({
    train: Dataset({
        features: ['claim', 'label', 'evidence', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 145449
    })
    validation: Dataset({
        features: ['claim', 'label', 'evidence', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9999
    })
    fever_test: Dataset({
        features: ['claim', 'label', 'evidence', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9999
    })
    pubhealth_test: Dataset({
        features: ['claim', 'evidence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1032
    })
    climate_test: Dataset({
        features: ['claim', 'label', 'evidence', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1381
    })
})


#### Setup model

In [18]:
from transformers import AutoModelForSequenceClassification

In [49]:
num_labels = 3 
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
print(model)

loading configuration file config.json from cache at /users/k21193529/.cache/huggingface/hub/models--bert-base-uncased/snapshots/1dbc166cf8765166998eff31ade2eb64c8a40076/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loa

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

#### Setup trainer

In [20]:
import numpy as np
import evaluate 
from transformers import TrainingArguments, Trainer

In [50]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references = labels)

In [51]:
batch_size = 16 #defaults to 8
num_epochs = 5

model_dir = model_dirs.get(experiment)
print(model_dir)

../models/BERT_FEVER


In [52]:
args = TrainingArguments(
    model_dir,
    evaluation_strategy = "epoch",
    # evaluation_strategy = "steps",
    save_strategy = "epoch",
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    load_best_model_at_end = True,
    # metric_for_best_model = "accuracy"
)   

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [53]:
trainer = Trainer(
    model, 
    args,
    train_dataset = encoded_ds["train"],
    eval_dataset = encoded_ds["validation"],
    tokenizer = tokenizer, 
    compute_metrics = compute_metrics
)

#### Train model

In [25]:
import torch

In [54]:
torch.cuda.empty_cache()

In [55]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claim, evidence. If claim, evidence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/scratch/users/k21193529/conda/jenv3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 145449
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 11365
  Number of trainable parameters = 109484547
/scratch/users/k21193529/conda/jenv3/lib/python3.8/site-packages/torch/nn/parallel/_funct

Epoch,Training Loss,Validation Loss,Accuracy
1,0.144600,0.146059,0.951395
2,0.096200,0.145917,0.954995
3,0.061500,0.171342,0.956496
4,0.042800,0.198486,0.956196
5,0.024700,0.216514,0.959296


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claim, evidence. If claim, evidence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9999
  Batch size = 64
Saving model checkpoint to ../models/BERT_FEVER/checkpoint-2273
Configuration saved in ../models/BERT_FEVER/checkpoint-2273/config.json
Model weights saved in ../models/BERT_FEVER/checkpoint-2273/pytorch_model.bin
tokenizer config file saved in ../models/BERT_FEVER/checkpoint-2273/tokenizer_config.json
Special tokens file saved in ../models/BERT_FEVER/checkpoint-2273/special_tokens_map.json
/scratch/users/k21193529/conda/jenv3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was a

TrainOutput(global_step=11365, training_loss=0.08048117621271189, metrics={'train_runtime': 5030.0515, 'train_samples_per_second': 144.58, 'train_steps_per_second': 2.259, 'total_flos': 1.9134791747559936e+17, 'train_loss': 0.08048117621271189, 'epoch': 5.0})

In [56]:
trainer.evaluate(encoded_ds['fever_test'])

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claim, evidence. If claim, evidence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9999
  Batch size = 64
/scratch/users/k21193529/conda/jenv3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.1847246289253235,
 'eval_accuracy': 0.9434943494349435,
 'eval_runtime': 27.0999,
 'eval_samples_per_second': 368.969,
 'eval_steps_per_second': 5.793,
 'epoch': 5.0}

In [57]:
trainer.evaluate(encoded_ds['pubhealth_test'])

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claim, evidence. If claim, evidence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1032
  Batch size = 64
/scratch/users/k21193529/conda/jenv3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.675694227218628,
 'eval_accuracy': 0.4932170542635659,
 'eval_runtime': 2.9031,
 'eval_samples_per_second': 355.482,
 'eval_steps_per_second': 5.856,
 'epoch': 5.0}

In [58]:
trainer.evaluate(encoded_ds['climate_test'])

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claim, evidence. If claim, evidence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1381
  Batch size = 64
/scratch/users/k21193529/conda/jenv3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 2.977294445037842,
 'eval_accuracy': 0.42867487328023174,
 'eval_runtime': 3.7283,
 'eval_samples_per_second': 370.411,
 'eval_steps_per_second': 5.901,
 'epoch': 5.0}

## Inference

In [21]:
# load best model
best_model_checkpoint = "../models/BERT_FEVER/checkpoint-4546"
best_model_checkpoint = "../models/BERT_PUBHEALTH/checkpoint-262"
best_model_checkpoint = "../models/BERT_CLIMATE/checkpoint-34"

In [ ]:
num_labels = 3 
model = AutoModelForSequenceClassification.from_pretrained(best_model_checkpoint, num_labels=num_labels)

In [ ]:
import numpy as np
from transformers import TrainingArguments, Trainer
import evaluate 

batch_size = 16 #defaults to 8
num_epochs = 5

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references = labels)

args = TrainingArguments(
    "../models/SciBERT_FEVER",
    evaluation_strategy = "epoch",
    # evaluation_strategy = "steps",
    save_strategy = "epoch",
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    load_best_model_at_end = True,
    # metric_for_best_model = "accuracy"
)   

trainer = Trainer(
    model, 
    args,
    # train_dataset = encoded_ds["train"],
    eval_dataset = encoded_ds["validation"],
    tokenizer = tokenizer, 
    compute_metrics = compute_metrics
)

In [ ]:
trainer.evaluate(encoded_ds['fever_test'])

In [ ]:
trainer.evaluate(encoded_ds['pubhealth_test'])

In [ ]:
trainer.evaluate(encoded_ds['climate'])